# Model 1: Fully Connected Neural Network

In [54]:
from pathlib import Path
import numpy as np
import torch 

In [55]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [56]:
# Import local modules from 'src/utils' as package 'utils'
import sys; sys.path.insert(0, '../')

In [57]:
import utils

## Read Datasets from .csv

In [58]:
from utils.file_io import read_angle_datasets

In [59]:
data_folder = Path("../../data/")
train_data, test_data = read_angle_datasets(data_folder, 0.9)

Reading .csv files: 1it [00:01,  1.45s/it]


In [60]:
input_shape, output_shape = train_data[0][0].shape[0], train_data[0][1].shape[0]
print(f"Data shape {input_shape} / {output_shape} of total {len(train_data)} data rows!")

Data shape 8 / 3 of total 62379 data rows!


/mnt/src/notebooks/../utils/angle_dataset.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return row[0], row[1]


## Model definition

In [61]:
from torch import nn, Tensor
from typing import Tuple, List

In [62]:
class FullyConnected(nn.Module):
    def __init__(self, flattened_input_dim: int, intermediate_dims: List[int], output_dim: int, dropout: float = 0.25, hidden_activation = nn.ReLU) -> None:
        super().__init__()
        self.total_epochs = 0
        self.flatten = nn.Flatten()
        self.hidden = nn.Sequential()
        for i, dim in enumerate(intermediate_dims):
            if i == 0:
                self.hidden.add_module(f"linear_{i+1}", nn.Linear(flattened_input_dim, dim))
            else:
                self.hidden.add_module(f"linear_{i+1}", nn.Linear(intermediate_dims[i-1], dim))

            self.hidden.add_module(f"hidden_activation_{i+1}", hidden_activation())
            self.hidden.add_module(f"dropout_{i+2}", nn.Dropout(dropout))

        self.last = nn.Linear(intermediate_dims[-1], output_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = self.flatten(x)
        x = self.hidden(x)
        return self.last(x)

## Load parameter, functions and dataloader

In [63]:
import os
from dotenv import load_dotenv
from torch.utils.data import DataLoader

from utils.file_io import save_model
from utils.evaluation import compute_loss_on
from utils.file_io import define_dataloader_from_angle_dataset

In [64]:
model_path = Path("../../models/fully_connected/").absolute()

In [65]:
dotenv_path = model_path / ".env"
load_dotenv(dotenv_path=dotenv_path)

learning_rate = float(os.getenv("LEARNING_RATE"))
batch_size = int(os.getenv("BATCH_SIZE"))
num_epochs = int(os.getenv("NUM_EPOCHS"))

In [66]:
def get_optimizer_function(model: nn.Module, learning_rate: float) -> torch.optim:
    return torch.optim.Adam(model.parameters(), lr=learning_rate)

In [67]:
def get_loss_function() -> nn.Module:
    return torch.nn.MSELoss()

In [68]:
train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_angle_dataset(train_data, test_data, batch_size, split_size=0.95)

## Define train methods

In [69]:
def train_epoch(train_dataloader: DataLoader, model: nn.Module, loss_function, optimizer, 
                device: torch.device, report_interval: int = 1000) -> float:

    running_loss = 0
    last_loss = 0
    
    for i, (inputs, true_values) in enumerate(train_dataloader):

        inputs = inputs.to(device)
        true_values = true_values.to(device)
                
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, true_values)
        running_loss += loss
        loss.backward()
        optimizer.step() 
    
    if i % report_interval == report_interval - 1:
        last_loss = running_loss / report_interval
        print(f"batch {i + 1}, Mean Squared Error: {last_loss}")
        running_loss = 0
    
    return last_loss 

In [70]:
def train(epochs: int, train_dataloader: DataLoader, validation_dataloader: DataLoader, model: nn.Module, loss_function, optimizer, 
          checkpoint_path: Path, device: torch.device = 'cpu', report_interval: int = 1000, tune: bool = False) -> nn.Module:

    best_val_loss = float("inf")

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    model.to(device)

    for epoch in range(model.total_epochs, epochs):
        print(f"Epoch: {epoch + 1}")

        model.train(True)
        avg_loss = train_epoch(train_dataloader, model, loss_function, optimizer, device, report_interval)
        model.eval()

        with torch.no_grad():
            avg_val_loss = compute_loss_on(validation_dataloader, model, loss_function, device=device)

        print(f"Loss on train: {avg_loss}, loss on validation: {avg_val_loss}")

        # Disable checkpoints while running in tune mode
        if avg_val_loss < best_val_loss and not tune:
            best_val_loss = avg_val_loss
            model_path = checkpoint_path / f"{checkpoint_path.name}_{epoch}.pt"
            save_model(model, model_path)

        model.total_epochs += 1
    
    return model   

## Train the model with grid search hyperparameter tuning

In [71]:
import ray
import json
from ray import tune
from ray import train as ray_train
from ray.train import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from typing import Dict
from utils.evaluation import compute_predictions, compute_losses_from
from functools import partial

In [72]:
def run_id_generator(data: Dict) -> str:
    return str(hash(json.dumps(data)))

In [73]:
def parameter_train(parameter: Dict, train_data: DataLoader, test_data: DataLoader, model_input_shape: int, model_output_shape: int, checkpoint_path: Path, device: torch.device) -> None:

    run_id = run_id_generator(parameter)
    run_checkpoint = checkpoint_path / run_id
    run_checkpoint.mkdir(parents=True)

    train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_angle_dataset(train_data, test_data, batch_size=parameter["batch_size"])

    model = FullyConnected(model_input_shape, parameter["hidden_layers"], model_output_shape)

    optimizer = get_optimizer_function(model, parameter["lr"])
    loss_function = get_loss_function()

    last_model = train(parameter["epochs"], train_dataloader, validation_dataloader, model, loss_function, optimizer, checkpoint_path, device, report_interval=50, tune=True)

    y, y_true = compute_predictions(test_dataloader, last_model)
    test_losses = compute_losses_from(y, y_true, loss_function)

    torch.save((last_model.state_dict(), optimizer.state_dict()), run_checkpoint / "checkpoint.pt")

    print(str(parameter), file=open(run_checkpoint / "trail_parameter.txt", "w"))

    ray_train.report(
        metrics={ "loss": float(test_losses.mean()), "parameter": parameter , "run_id": run_id }
    )

Define parameter ranges

In [74]:
learning_rate_radius = 1e-3
num_epochs_radius = 50
batch_size_radius = 10
hidden_layer_sizes = [16, 32, 64]
hidden_layer_count = [2, 4, 6]

In [75]:
parameter = {
    "lr": tune.loguniform(learning_rate_radius - learning_rate, learning_rate + learning_rate_radius),
    "epochs": tune.choice(list(range(num_epochs - num_epochs_radius, num_epochs + num_epochs_radius, 20))),
    "batch_size": tune.grid_search(list(range(batch_size - batch_size_radius, batch_size + batch_size_radius, 4))),
    "hidden_layers": tune.grid_search(list([hidden_layer_sizes[i % len(hidden_layer_sizes)] for i in range(dim)] for _, dim in enumerate(hidden_layer_count))),
    "dropout": tune.uniform(0.1, 0.5)
}

In [76]:
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=num_epochs,
    grace_period=1,
    reduction_factor=2,
)

In [77]:
if ray.is_initialized():
    ray.shutdown()

ray.init(runtime_env={ "py_modules": [utils] })

2023-11-19 17:55:45,948	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-11-19 17:55:46,057	INFO worker.py:1673 -- Started a local Ray instance.
2023-11-19 17:55:46,063	INFO packaging.py:530 -- Creating a file package for local directory '/mnt/src/notebooks/../utils'.
2023-11-19 17:55:46,066	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_d8c8a9358aefbc3b.zip' (0.03MiB) to Ray cluster...
2023-11-19 17:55:46,067	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_d8c8a9358aefbc3b.zip'.


Python version:,3.10.12
Ray version:,2.8.0


In [78]:
ray_resources_manager = tune.with_resources(
    trainable=lambda param: parameter_train(param, train_data, test_data, input_shape, output_shape, model_path, device),
    # See: https://stackoverflow.com/questions/58967793/what-is-the-way-to-make-tune-run-parallel-trials-across-multiple-gpus
    resources={ "cpu": 3, "gpu": 0.25 }
)

tuner = tune.Tuner(
    ray_resources_manager,
    param_space=parameter,
    tune_config=tune.TuneConfig(
        scheduler=scheduler,
        num_samples=20
    )
)

In [79]:
results = tuner.fit()

2023-11-19 17:56:07,158	WARNING worker.py:2074 -- Warning: The actor ImplicitFunc is very large (15 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


(<lambda> pid=25144) Epoch: 1


(<lambda> pid=25144) /tmp/ray/session_2023-11-19_17-55-44_162449_23591/runtime_resources/py_modules_files/_ray_pkg_d8c8a9358aefbc3b/utils/angle_dataset.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
(<lambda> pid=25144)   return row[0], row[1]


(<lambda> pid=25144) Loss on train: 0, loss on validation: 0.5568861961364746
(<lambda> pid=25144) Epoch: 2 [repeated 4x across cluster]


In [ ]:
best_result = results.get_best_result("loss", "min")

for key in best_result.metrics["parameter"]:
    print(f"Best Trail: {key} with {best_result.metrics['parameter'][key]}")

In [ ]:
# Checkpoint contains model and optimizer function
best_model, _ = torch.load(model_path / best_result.metrics["run_id"] / "checkpoint.pt")

In [ ]:
print(f"Currently best trial defined by loss: {best_result.metrics['run_id']}", file=open(model_path / "best_trail.txt", "w"))

## Evaluation

In [ ]:
from utils.visualization import create_trace_animation
from matplotlib import pyplot as plt
from IPython.display import HTML

In [ ]:
# Compute eval on the cpu
device = 'cpu'

In [ ]:
%matplotlib notebook
 
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

### Loading the best model

In [ ]:
from utils.file_io import load_model

In [ ]:
loss_function = get_loss_function()

In [ ]:
model = FullyConnected(input_shape, best_result.metrics["parameter"]["hidden_layers"], output_shape)
model.load_state_dict(best_model)
model.eval()

In [ ]:
y, y_true = compute_predictions(test_dataloader, model, device)
test_losses = compute_losses_from(y, y_true, loss_function)
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")

In [ ]:
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())